In [1]:
import numpy as np

def parse_pose_file(pose_file):
    cam_K = None
    cam_RT = None
    lidar_R = None
    lidar_T = None
    
    with open(pose_file, 'r') as f:
        for line in f:
            line = line.strip()
            
            # Check and parse each line for cam_K, cam_RT, lidar_R, and lidar_T
            if line.startswith('cam_K:'):
                cam_K = np.array([float(x) for x in line.split(' ')[1:]]).reshape(3, 3)
                
            elif line.startswith('cam_RT:'):
                cam_RT = np.array([float(x) for x in line.split(' ')[1:]]).reshape(4, 4)
                
            elif line.startswith('lidar_R:'):
                lidar_R = np.array([float(x) for x in line.split(' ')[1:]]).reshape(3, 3)
                
            elif line.startswith('lidar_T:'):
                lidar_T = np.array([float(x) for x in line.split(' ')[1:]])

    return cam_K, cam_RT, lidar_R, lidar_T

# Example usage
pose_file = '/project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/calib/1624045591761.txt'  # Replace with the actual path to your pose file
cam_K, cam_RT, lidar_R, lidar_T = parse_pose_file(pose_file)

print("Camera Intrinsic Matrix (cam_K):")
print(cam_K)

print("\nCamera Extrinsic Matrix (cam_RT):")
print(cam_RT)

print("\nLidar Rotation Matrix (lidar_R):")
print(lidar_R)

print("\nLidar Translation Vector (lidar_T):")
print(lidar_T)


Camera Intrinsic Matrix (cam_K):
[[1.48775283e+03 0.00000000e+00 6.25011716e+02]
 [0.00000000e+00 1.47106344e+03 3.76483801e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Camera Extrinsic Matrix (cam_RT):
[[ 0.99902424 -0.03870134 -0.02127828  0.00191298]
 [-0.02560896 -0.11508902 -0.99302502  0.14080302]
 [ 0.03598251  0.99260099 -0.11596782  0.08308956]
 [ 0.          0.          0.          1.        ]]

Lidar Rotation Matrix (lidar_R):
[[-0.996479   -0.0834525   0.00805512]
 [ 0.0837999  -0.994374    0.0647862 ]
 [ 0.00260323  0.0652331   0.997867  ]]

Lidar Translation Vector (lidar_T):
[0. 0. 0.]


In [ ]:
import os
import numpy as np

def parse_cam_RT(pose_file):
    cam_RT = None
    with open(pose_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('cam_RT:'):
                cam_RT = np.array([float(x) for x in line.split(' ')[1:]]).reshape(4, 4)
                break  # Once cam_RT is found, we can stop reading
    return cam_RT

def process_pose_files(input_folder, output_file):
    # Get all .txt files in the input folder and sort by file name
    txt_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.txt')])
    
    # Open the output file in write mode
    with open(output_file, 'w') as out_f:
        # Iterate over all sorted txt files
        for txt_file in txt_files:
            txt_file_path = os.path.join(input_folder, txt_file)
            
            # Extract cam_RT matrix from the current file
            cam_RT = parse_cam_RT(txt_file_path)
            
            if cam_RT is not None:
                # Write the filename and cam_RT matrix to the output file
                out_f.write(f'File: {txt_file}\n')
                np.savetxt(out_f, cam_RT, fmt='%.6f')  # Save cam_RT matrix with 6 decimal points
                out_f.write('\n')  # Add a blank line after each matrix for readability
            else:
                print(f"No cam_RT found in {txt_file}")
    
    print(f"All cam_RT matrices saved to {output_file}")

# Example usage
input_folder = '/project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/calib/'  # Replace with the actual folder path containing .txt files
output_file = 'poses.txt'  # The file where you want to save the cam_RT matrices

process_pose_files(input_folder, output_file)


In [ ]:
import os
import numpy as np

def parse_cam_RT(pose_file):
    cam_RT = None
    with open(pose_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('cam_RT:'):
                cam_RT = np.array([float(x) for x in line.split(' ')[1:]]).reshape(4, 4)
                break  # Exit the loop after finding cam_RT
    return cam_RT

def process_pose_files(input_folder, output_file):
    #Get all .txt files in the folder and sort them by file name
    txt_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.txt')])

    # Open the output file in write mode
    with open(output_file, 'w') as out_f:
        # Traverse all txt files
        for txt_file in txt_files:
            txt_file_path = os.path.join(input_folder, txt_file)
            
            # Extract the cam_RT matrix from the current file
            cam_RT = parse_cam_RT(txt_file_path)
            
            if cam_RT is not None:
                # Flatten cam_RT into one line
                flattened_cam_RT = cam_RT.flatten()
                
                # Remove the last four numbers
                truncated_cam_RT = flattened_cam_RT[:-4]
                
                # Print the shape after removing the last four numbers
                print(f"Shape of truncated cam_RT from {txt_file}: {truncated_cam_RT.shape}")
                
                # Write the cam_RT data with the last four numbers removed to the output file
                out_f.write(' '.join([f'{x:.6f}' for x in truncated_cam_RT]))
                out_f.write('\n')  # The newline character separates the pose information of each frame
            else:
                print(f"No cam_RT found in {txt_file}")
    
    print(f"All cam_RT matrices saved to {output_file}")

# Example usage
input_folder = '/project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/validation/calib'  # Replace with the folder path containing the .txt file
output_file = 'poses.txt' 

process_pose_files(input_folder, output_file)


In [12]:
import numpy as np

def check_bin_file(file_name):
    points = np.fromfile(file_name, dtype=np.float32)
    print(f"File: {file_name}")
    print(f"Total points: {len(points)}")
    print(f"First 10 values: {points[:10]}")

# Replace with the actual file path
check_bin_file('/project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/1624045591761.bin')


File: /project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/1624045591761.bin
Total points: 554570
First 10 values: [ 21.10134   -21.105757    3.124189    1.         26.         20.947523
 -20.95191     2.5858066   2.          8.       ]


In [1]:
import os
import shutil

# Define path
source_dir = '/project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data'
target_dir = os.path.join(source_dir, '00000')

# If the target folder does not exist, create it
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Traverse all files in the source folder
for filename in os.listdir(source_dir):
    # Only process .bin files
    if filename.endswith('.bin'):
        # Build the complete source file path and target file path
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)
        
        # Move file
        shutil.move(source_file, target_file)
        print(f'Moved: {filename} to {target_dir}')

print("All .bin files have been moved.")


Moved: 1624045564360.bin to /project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/00000
Moved: 1624045564461.bin to /project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/00000
Moved: 1624045564562.bin to /project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/00000
Moved: 1624045564662.bin to /project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/00000
Moved: 1624045564761.bin to /project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/00000
Moved: 1624045564863.bin to /project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/00000
Moved: 1624045564961.bin to /project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/00000
Moved: 1624045565065.bin to /project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset/training/lidar_data/00000
Moved: 1624045565162.bin to /pro

In [2]:
import os
import pandas as pd

def check_pose_and_bin_files(dataset_dir):
    # Define training, validation, and testing set folders
    sequences = ['training', 'validation', 'testing']
    
    for sequence in sequences:
        sequence_dir = os.path.join(dataset_dir, sequence)
        lidar_data_dir = os.path.join(sequence_dir, 'lidar_data', '00000')
        poses_file = os.path.join(sequence_dir, 'poses.txt')

        # Check if the lidar_data folder and poses.txt file exist
        if not os.path.exists(lidar_data_dir):
            print(f"Lidar data folder not found: {lidar_data_dir}")
            continue
        if not os.path.exists(poses_file):
            print(f"Poses.txt file not found: {poses_file}")
            continue
        
        # Read the poses in the poses.txt file
        gt_transforms = pd.read_csv(poses_file, header=None, sep=' ')
        pose_count = len(gt_transforms)

        # Get the number of .bin files
        bin_files = [f for f in os.listdir(lidar_data_dir) if f.endswith('.bin')]
        bin_count = len(bin_files)
        
        # Print inspection results
        print(f"Sequence: {sequence}")
        print(f"Number of poses in poses.txt: {pose_count}")
        print(f"Number of .bin files in lidar_data/00000: {bin_count}")

        # Check if the quantity matches
        if pose_count == bin_count:
            print("Pose count matches bin file count.")
        else:
            print(f"Mismatch: {pose_count} poses vs {bin_count} bin files.")
        print("-" * 50)

if __name__ == '__main__':
    # Specify the path of the dataset
    dataset_dir = '/project/bli4/autoai/joyang/GeoTransformer/data/ORFD/Final_Dataset'
    check_pose_and_bin_files(dataset_dir)


Sequence: training
Number of poses in poses.txt: 8392
Number of .bin files in lidar_data/00000: 8392
Pose count matches bin file count.
--------------------------------------------------
Sequence: validation
Number of poses in poses.txt: 1245
Number of .bin files in lidar_data/00000: 1245
Pose count matches bin file count.
--------------------------------------------------
Sequence: testing
Number of poses in poses.txt: 2193
Number of .bin files in lidar_data/00000: 2193
Pose count matches bin file count.
--------------------------------------------------
